Leia-Me: Script de Processamento de Arquivos

O que o script faz?

->Procura subpastas que começam com "BK_".

->Renomeia arquivos que começam com "df" (adiciona a data de modificação no nome).
-.Move os arquivos renomeados para uma pasta chamada "Dados_Tratados".
->Registra tudo isso num arquivo CSV chamado "rel_cargas.csv".
->Ignora arquivos duplicados.

Antes de usar
Instale as bibliotecas necessárias:
pip install pandas

Organize as pastas:
->Base: Coloque os arquivos em C:\Users\Admin\Desktop\BACKUP (ou outro caminho, é só ajustar no código).
->Saída: A pasta Dados_Tratados será criada automaticamente.
->Como usar?
->Coloque os arquivos nas subpastas que começam com BK_.
->Ajuste o caminho base no código, se necessário.
!!Execute o script.


#Importação de bibliotecas e definição de caminhos
import os
import pandas as pd
from datetime import datetime
import shutil

# Caminhos das pastas (pode ser alterado)
caminho_base = r'C:\Users\Admin\Desktop\BACKUP'  # Caminho base para os arquivos
caminho_dados_tratados = os.path.join(caminho_base, 'Dados_Tratados')  # Pasta de arquivos tratados
caminho_rel_cargas = os.path.join(caminho_base, 'rel_cargas', 'rel_cargas.csv')  # Arquivo de registro

In [20]:
import os
import pandas as pd
from datetime import datetime
import shutil

# Caminhos das pastas (pode ser alterado)
caminho_base = r'C:\Users\Admin\Desktop\BACKUP'  # Caminho base para os arquivos
caminho_dados_tratados = os.path.join(caminho_base, 'Dados_Tratados')  # Pasta de arquivos tratados
caminho_rel_cargas = os.path.join(caminho_base, 'rel_cargas', 'rel_cargas.csv')  # Arquivo de registro


Função para obter a data de modificação do arquivo

In [21]:
def obter_data_modificacao(arquivo_path):
    """
    Retorna a data de modificação do arquivo no formato 'YYYY-MM-DD'.
    """
    timestamp = os.path.getmtime(arquivo_path)
    return datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')

 Verificar ou criar arquivo de registro

In [22]:
if os.path.exists(caminho_rel_cargas):  # Arquivo de registro (pode ser alterado)
    # Carregar o registro existente
    registro_df = pd.read_csv(caminho_rel_cargas)
else:
    # Criar um DataFrame vazio se o registro não existir
    registro_df = pd.DataFrame(columns=['Nome Arquivo', 'Data Modificação'])

Processar pastas que começam com 'BK_

In [23]:
for nome_pasta in os.listdir(caminho_base):  # Caminho base (pode ser alterado)
    caminho_pasta = os.path.join(caminho_base, nome_pasta)
    if os.path.isdir(caminho_pasta) and nome_pasta.startswith('BK_'):
        # Obter a lista de arquivos na subpasta
        arquivos_na_pasta = [f for f in os.listdir(caminho_pasta) if os.path.isfile(os.path.join(caminho_pasta, f))]
        
        # Lista para armazenar novos registros
        novos_registros = []

        # Processamento de arquivos na subpasta
        for arquivo in arquivos_na_pasta:
            # Verificar se o nome do arquivo começa com 'df' (maiúsculo ou minúsculo)
            if arquivo.lower().startswith('df'):
                caminho_arquivo = os.path.join(caminho_pasta, arquivo)
                data_modificacao = obter_data_modificacao(caminho_arquivo)
                nome_arquivo_modificado = f"{os.path.splitext(arquivo)[0]}_{data_modificacao}{os.path.splitext(arquivo)[1]}"
                
                # Verificar se o arquivo já está no registro
                if not ((registro_df['Nome Arquivo'] == nome_arquivo_modificado) & (registro_df['Data Modificação'] == data_modificacao)).any():
                    # Adicionar novo registro
                    novos_registros.append({'Nome Arquivo': nome_arquivo_modificado, 'Data Modificação': data_modificacao})
                    # Mover e renomear o arquivo para a pasta de dados tratados
                    caminho_destino_arquivo = os.path.join(caminho_dados_tratados, nome_arquivo_modificado)  # Pasta tratada (pode ser alterado)
                    shutil.move(caminho_arquivo, caminho_destino_arquivo)
                    print(f"Arquivo movido e renomeado para: {caminho_destino_arquivo}")
                else:
                    print(f"Arquivo duplicado identificado e não movido: {nome_arquivo_modificado}")

        # Atualização do registro
        if novos_registros:
            novos_registros_df = pd.DataFrame(novos_registros)
            registro_df = pd.concat([registro_df, novos_registros_df], ignore_index=True)
            # Criar o diretório de registro, se não existir
            os.makedirs(os.path.dirname(caminho_rel_cargas), exist_ok=True)
            # Salvar o registro atualizado
            registro_df.to_csv(caminho_rel_cargas, index=False)  # Arquivo de registro (pode ser alterado)
            print(f"{len(novos_registros)} novos arquivos adicionados ao registro.")
        else:
            print("Nenhum novo arquivo para adicionar ao registro.")


Arquivo duplicado identificado e não movido: dfPostgreSQL_2024-10-02.csv
Arquivo duplicado identificado e não movido: df_db2_2024-10-02.csv
Arquivo duplicado identificado e não movido: df_Mysql_2024-10-02.csv
Arquivo duplicado identificado e não movido: df_Oracle_2024-10-02.csv
Arquivo duplicado identificado e não movido: df_SqlServer_2024-10-02.csv
Arquivo duplicado identificado e não movido: df_teradata_2024-10-02.csv
Nenhum novo arquivo para adicionar ao registro.
Arquivo duplicado identificado e não movido: dfPostgreSQL_2024-11-04.csv
Arquivo duplicado identificado e não movido: df_db2_2024-11-04.csv
Arquivo duplicado identificado e não movido: df_Mysql_2024-11-04.csv
Arquivo duplicado identificado e não movido: df_Oracle_2024-11-04.csv
Arquivo duplicado identificado e não movido: df_SqlServer_2024-11-04.csv
Arquivo duplicado identificado e não movido: df_teradata_2024-11-04.csv
Nenhum novo arquivo para adicionar ao registro.
Arquivo duplicado identificado e não movido: dfPostgreSQ

Atualização do registro

    
        if novos_registros:
            novos_registros_df = pd.DataFrame(novos_registros)
            registro_df = pd.concat([registro_df, novos_registros_df], ignore_index=True)
            # Criar o diretório de registro, se não existir
            os.makedirs(os.path.dirname(caminho_rel_cargas), exist_ok=True)
            # Salvar o registro atualizado
            registro_df.to_csv(caminho_rel_cargas, index=False)  # Arquivo de registro (pode ser alterado)
            print(f"{len(novos_registros)} novos arquivos adicionados ao registro.")
        else:
            print("Nenhum novo arquivo para adicionar ao registro.")